# Installation

In [ ]:
# pip install pandas transformers 

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [20]:
import torch

print("torch + cuda:", torch.__version__) # Check PyTorch version
print("Is cuda avialable:", torch.cuda.is_available())

torch + cuda: 2.6.0+cu124
Is cuda avialable: True


# Dataset Pre-processing

In [21]:
DATASET = "FoCus"

In [22]:
import pandas as pd
import json

with open(f'./datasets/{DATASET}/train.json') as f:
    valid_data = json.load(f)

In [23]:
def convertToDialogue(my_list):
    formatted_string = ""
    for index, item in enumerate(my_list):
        if index % 2 == 0:
            user = "User1"
        else:
            user = "User2"
        formatted_string += f"{user}: {item}\n"
    formatted_string = formatted_string.rstrip("\n")
    return formatted_string

flattened_data = []
data_list = valid_data['data']
for entry in data_list:
    persona =  "".join(entry['persona'])
    list_length = len(entry["utterance"])
    last_utterance = entry["utterance"][-1]
    dialogue_key = f"dialogue{list_length}"
    last_item = last_utterance[dialogue_key]
    flattened_data.append({
                'dialogID': entry['dialogID'],
                'persona': persona,
                'utterance': convertToDialogue(last_item)
            })

df = pd.DataFrame(flattened_data)

In [24]:
df = df.replace(r'\*\*', '', regex=True)
df = df.replace(r'\r', '', regex=True)
df = df.replace("'", "", regex=True)

df.dropna(inplace=True)

# Function to split the conversation
def split_conversation(conv_str):
    utterances = conv_str.split("\n")
    context = "\n".join(utterances[:-1])
    response = utterances[-1]
    return context, response

new_rows = []
for index, row in df.iterrows():
    context, response = split_conversation(row['utterance'])
    new_row = {
        'personas': row['persona'],
        'context': context,
        'act_response': response
    }
    new_rows.append(new_row)

new_df = pd.DataFrame(new_rows)

new_df.head(4)

,personas,context,act_response
0,I like to go to Church.I am Roman Catholic.I w...,"User1: Wow, this is amazing! What is this?\nUs...",User2: It is in Texas state.
1,I like living in a city.I dont hope to ever vi...,"User1: I know this place, but I dont remember ...","User2: Of course, Captain William Cornwallis S..."
2,I want to visit Mexico.I am interested in the ...,"User1: I know this place, but I dont remember ...","User2: Well, in this rainforest, especially th..."
3,I am afraid of bears.I like valleys.I live in ...,User1: Where is this place?\nUser2: This place...,"User2: Not only it, but there are various othe..."


In [25]:
# Calculate minimum and maximum number of words in each column
min_persona_length = new_df['personas'].apply(lambda x: len(x.split())).min()
max_persona_length = new_df['personas'].apply(lambda x: len(x.split())).max()

min_context_length = new_df['context'].apply(lambda x: len(x.split())).min()
max_context_length = new_df['context'].apply(lambda x: len(x.split())).max()

min_response_length = new_df['act_response'].apply(lambda x: len(x.split())).min()
max_response_length = new_df['act_response'].apply(lambda x: len(x.split())).max()

# Print the lengths in min-max format
print(f"Persona Length (in words): {min_persona_length}-{max_persona_length}")
print(f"Context Length (in words): {min_context_length}-{max_context_length}")
print(f"Response Length (in words): {min_response_length}-{max_response_length}")

Persona Length (in words): 11-92
Context Length (in words): 46-705
Response Length (in words): 2-159


In [26]:
# Save the new DataFrame to a CSV file
new_df.to_csv(f'./datasets/{DATASET}/ds_cleaned.csv', index=False)

# PAA

**The model consists of the following key components:**

- Persona Encoder - Applies self-attention to persona embeddings.
- Context Encoder - Applies self-attention to dialogue history embeddings.
- Persona-Adaptive Attention (PAA) - Uses cross-attention between persona and context, dynamically adjusting their influence.
- Dialogue Decoder - Generates responses using a transformer-based model, incorporating persona-aware representations.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/home/salehafzoon/Desktop/PAA_Implementation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET = "FoCus"
BATCH_SIZE = 16
EPOCS = 100
EMBEDDING_SIZE = 768
MAX_LENGTH_TOKEN = 50

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer


csv_path = f"datasets/{DATASET}/ds_cleaned.csv"

df = pd.read_csv(csv_path)

assert "personas" in df.columns and "context" in df.columns and "act_response" in df.columns, "Missing columns in dataset!"


In [4]:
df.head()

,personas,context,act_response
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,User2: The history of the house you are intere...
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",User2: This house was use as a stop for slaves...
2,I am fascinated by the Spanish Colonial Reviva...,"User1: Wow, this is amazing! What is this?\nUs...","User2: Sure, you will like to know that this p..."
3,I want to become a college student.I want to s...,User1: Where is this place?\nUser2: Hello! Wel...,User2: Technische Universität Darmstadt in the...
4,I like to visit england.I love church.I would ...,User1: Where is this place?\nUser2: This place...,"User2: I suggest a place, for your wish of see..."


In [5]:
class SelfAttention(nn.Module):
    """Single-head self-attention mechanism."""
    def __init__(self, embed_size):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(embed_size, embed_size)
        self.key = nn.Linear(embed_size, embed_size)
        self.value = nn.Linear(embed_size, embed_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.shape[-1] ** 0.5)
        attention_weights = self.softmax(attention_scores)

        output = torch.matmul(attention_weights, V)
        return output


In [6]:
class PersonaEncoder(nn.Module):
    def __init__(self, embed_size):
        super(PersonaEncoder, self).__init__()
        self.attention = SelfAttention(embed_size)
        self.norm = nn.LayerNorm(embed_size)

    def forward(self, persona_embeds):
        attended = self.attention(persona_embeds)
        return self.norm(attended + persona_embeds)  # Residual Connection


In [7]:
class ContextEncoder(nn.Module):
    def __init__(self, embed_size):
        super(ContextEncoder, self).__init__()
        self.attention = SelfAttention(embed_size)
        self.norm = nn.LayerNorm(embed_size)

    def forward(self, context_embeds):
        attended = self.attention(context_embeds)
        return self.norm(attended + context_embeds)  # Residual Connection


In [8]:
class PersonaAdaptiveAttention(nn.Module):
    def __init__(self, embed_size):
        super(PersonaAdaptiveAttention, self).__init__()
        self.context_attention = SelfAttention(embed_size)
        self.persona_attention = SelfAttention(embed_size)

        self.weighting = nn.Linear(embed_size, 1)  # Adaptive weighting
        self.norm = nn.LayerNorm(embed_size)

    def forward(self, persona_embeds, context_embeds):
        attended_context = self.context_attention(context_embeds)
        attended_persona = self.persona_attention(persona_embeds)

        # Adaptive Weighting (From PAA Mechanism)
        weights = torch.sigmoid(self.weighting(attended_persona))
        weighted_persona = weights * attended_persona

        # Fusion: Combining Persona and Context
        fused_representation = attended_context + weighted_persona
        return self.norm(fused_representation)


In [9]:
class DialogueDecoder(nn.Module):
    def __init__(self, model_name="gpt2", embed_size = EMBEDDING_SIZE):
        super(DialogueDecoder, self).__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained(model_name)

    def forward(self, input_embeds, response_tokens):
        outputs = self.gpt2(inputs_embeds=input_embeds, labels=response_tokens)
        return outputs.loss, outputs.logits


In [10]:
class PersonaAdaptiveChatbot(nn.Module):
    def __init__(self, embed_size = EMBEDDING_SIZE, model_name="gpt2"):
        super(PersonaAdaptiveChatbot, self).__init__()

        self.persona_encoder = PersonaEncoder(embed_size)
        self.context_encoder = ContextEncoder(embed_size)
        self.paa = PersonaAdaptiveAttention(embed_size)
        self.decoder = DialogueDecoder(model_name, embed_size)

        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    def forward(self, persona_tokens, context_tokens, response_tokens):
        # Token Embeddings
        persona_embeds = self.decoder.gpt2.transformer.wte(persona_tokens)
        context_embeds = self.decoder.gpt2.transformer.wte(context_tokens)

        # Encoders
        encoded_persona = self.persona_encoder(persona_embeds)
        encoded_context = self.context_encoder(context_embeds)

        # Persona-Adaptive Attention
        fused_representation = self.paa(encoded_persona, encoded_context)

        # Use PAA Output for GPT-2 Decoder
        response_embeds = self.decoder.gpt2.transformer.wte(response_tokens)
        response_embeds = response_embeds + fused_representation  # Inject PAA info

        return self.decoder(response_embeds, response_tokens)


In [11]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 does not have a padding token

# Tokenization function
def tokenize_texts(persona, context, response, max_length = MAX_LENGTH_TOKEN):
    persona_tokens = tokenizer(persona, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")["input_ids"]
    context_tokens = tokenizer(context, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")["input_ids"]
    response_tokens = tokenizer(response, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")["input_ids"]
    return persona_tokens.squeeze(0), context_tokens.squeeze(0), response_tokens.squeeze(0)

# Apply tokenization to the entire dataset
df["tokenized"] = df.apply(lambda row: tokenize_texts(row["personas"], row["context"], row["act_response"]), axis=1)

# Extract tokenized tensors
persona_tensors = torch.stack([x[0] for x in df["tokenized"]])
context_tensors = torch.stack([x[1] for x in df["tokenized"]])
response_tensors = torch.stack([x[2] for x in df["tokenized"]])


In [12]:
class FoCusDataset(Dataset):
    def __init__(self, persona_tensors, context_tensors, response_tensors):
        self.persona_tensors = persona_tensors
        self.context_tensors = context_tensors
        self.response_tensors = response_tensors

    def __len__(self):
        return len(self.persona_tensors)

    def __getitem__(self, idx):
        return {
            "personas": self.persona_tensors[idx],
            "context": self.context_tensors[idx],
            "response": self.response_tensors[idx],
        }

# Create dataset
dataset = FoCusDataset(persona_tensors, context_tensors, response_tensors)

# Create DataLoader
train_loader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)


In [13]:
# Define model
model = PersonaAdaptiveChatbot()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Define loss function (GPT-2 uses CrossEntropyLoss)
criterion = nn.CrossEntropyLoss()


In [14]:
MODEL_SAVE_DIR = f"trained_model/{DATASET}"

device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [15]:
import torch
from tqdm import tqdm
import os

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, dataloader, optimizer, num_epochs = EPOCS):
    model.to(device)  # Move model to GPU
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

        for batch in progress_bar:
            # Move batch tensors to GPU
            persona_tokens = batch["personas"].to(device)
            context_tokens = batch["context"].to(device)
            response_tokens = batch["response"].to(device)

            optimizer.zero_grad()
            loss, logits = model(persona_tokens, context_tokens, response_tokens)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update tqdm progress bar with current loss
            progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

    # Save the model after each epoch
    model_save_path = os.path.join(MODEL_SAVE_DIR, f"model_epoch_{EPOCS}.pth")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at: {model_save_path}")


In [16]:
# Train the model with CSV data
train(model, train_loader, optimizer)


Epoch 1/100: 100%|██████████| 63/63 [00:03<00:00, 20.49it/s, loss=4.88]


Epoch 1/100, Loss: 302.6696


Epoch 2/100: 100%|██████████| 63/63 [00:03<00:00, 20.78it/s, loss=3.91]


Epoch 2/100, Loss: 238.2913


Epoch 3/100: 100%|██████████| 63/63 [00:03<00:00, 20.81it/s, loss=3.11]


...

Epoch 98/100, Loss: 0.5189


Epoch 99/100: 100%|██████████| 63/63 [00:03<00:00, 18.70it/s, loss=0.00767]


Epoch 99/100, Loss: 0.4830


Epoch 100/100: 100%|██████████| 63/63 [00:03<00:00, 18.65it/s, loss=0.00712]


Epoch 100/100, Loss: 0.4488
Model saved at: trained_model/FoCus/model_epoch_100.pth


# Response Generation

In [17]:
DATASET = "FoCus"
MODEL_SAVE_DIR = f"trained_model/{DATASET}"
EPOCS = 100
MAX_LENGTH_TOKEN = 50

In [18]:
model = PersonaAdaptiveChatbot()  # Recreate the model architecture
model.load_state_dict(torch.load(f"{MODEL_SAVE_DIR}/model_epoch_{EPOCS}.pth"))
model.to(device)


PersonaAdaptiveChatbot(
  (persona_encoder): PersonaEncoder(
    (attention): SelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (context_encoder): ContextEncoder(
    (attention): SelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (paa): PersonaAdaptiveAttention(
    (context_attention): SelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (

In [19]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_response(model, persona_text, context_text, max_length= MAX_LENGTH_TOKEN):
    model.to(device)  # Ensure model is on GPU
    model.eval()

    # Tokenize and move to GPU, ensuring padding for alignment
    persona_tokens = tokenizer(persona_text, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)
    context_tokens = tokenizer(context_text, return_tensors="pt", padding=True, truncation=True)["input_ids"].to(device)

    with torch.no_grad():
        persona_embeds = model.decoder.gpt2.transformer.wte(persona_tokens)
        context_embeds = model.decoder.gpt2.transformer.wte(context_tokens)

        # Determine max sequence length for padding
        max_seq_len = max(persona_embeds.shape[1], context_embeds.shape[1])

        # Pad tensors to the same length
        pad_size_persona = max_seq_len - persona_embeds.shape[1]
        pad_size_context = max_seq_len - context_embeds.shape[1]

        persona_embeds = torch.nn.functional.pad(persona_embeds, (0, 0, 0, pad_size_persona), "constant", 0)
        context_embeds = torch.nn.functional.pad(context_embeds, (0, 0, 0, pad_size_context), "constant", 0)

        # Encode using persona and context encoders
        encoded_persona = model.persona_encoder(persona_embeds)
        encoded_context = model.context_encoder(context_embeds)

        # Apply Persona-Adaptive Attention (PAA)
        fused_representation = model.paa(encoded_persona, encoded_context)

    # Generate response ensuring input is on GPU
    generated = model.decoder.gpt2.generate(input_ids=context_tokens, max_length=max_length).to(device)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# Example
persona_text = "I love sci-fi movies."
context_text = "user1: What's your favorite movie?"
response = generate_response(model, persona_text, context_text)
print("Generated Response:", response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Response: user1: What's your favorite movie?
